## Langchain Expression Language Basics
- Langchain Expression Language is that **two runnable** can be **"chained" together** into sequences.
- The **output of the previous runnable.invoke()** call is passed **as input to the next runnable**.
- This can be done using the pipe operator **|**, or the more explicit **.pipe()** method, which does the same thing.

### Type of LCEL Chains
1. Sequential Chain
2. Chain Runnables
3. Parallel Chain
4. Router Chain
5. Custom Chain (Runnable Sequence)

In [45]:
from dotenv import load_dotenv

load_dotenv("./../.env")

True

In [46]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)

In [47]:
base_url = os.getenv("OLLAMA_BASE_URL")
model = "llama3.2:latest"

llm = ChatOllama(
    base_url=base_url,
    model=model
)
llm

ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')

## Without Chain

In [48]:
system_template = SystemMessagePromptTemplate.from_template("You are {school} teacher. You answer in short sentences")
question_template = HumanMessagePromptTemplate.from_template("Tell me about {topics} in {points} points")

messages = [system_template, question_template]
template = ChatPromptTemplate(messages=messages)

question = template.invoke({
    "school": "primary",
    "topics": "solar",
    "points": 5,
})

response = llm.invoke(question)
print(response.content)

Here's information about solar in 5 points:

1. Solar energy is a renewable source of power.
2. It's generated from the sun's rays, converted into electricity.
3. Solar panels convert sunlight into electrical energy.
4. We can use solar energy to power homes, schools, and communities.
5. It's a clean and sustainable way to reduce our reliance on fossil fuels.


## Sequential Chain

In [49]:
chain = template | llm

response = chain.invoke({
    "school": "primary",
    "topics": "solar",
    "points": 5,
})

print(response.content)

Here are 5 key points about solar:

1. Solar energy is power from the sun.
2. We use solar panels to catch sunlight and turn it into electricity.
3. Solar energy helps our planet by reducing greenhouse gases.
4. It's a clean and renewable source of energy, never running out.
5. We can use solar energy for homes, schools, and even our classrooms!


In [50]:
response = chain.invoke({
    "school": "pdh",
    "topics": "solar",
    "points": 5,
})

print(response.content)

Here are 5 key points about solar:

1. Solar energy is renewable and sustainable.
2. It harnesses power from the sun's rays to generate electricity or heat.
3. Solar panels convert sunlight into DC power, which can be used or stored.
4. Solar energy is zero-emission, producing no greenhouse gas emissions or pollution.
5. Global solar energy capacity has grown significantly in recent years, becoming a major source of clean energy worldwide.


### StrOutputParser
- Using StrOutputParser will create a final output as the string, so that we can pass this string data to some other runnable if we want

In [51]:
from langchain_core.output_parsers import StrOutputParser

chain = template | llm | StrOutputParser()

response = chain.invoke({
    "school": "pdh",
    "topics": "solar",
    "points": 5,
})

print(response)


Here are 5 key points about solar:

1. Solar energy is generated from sunlight.
2. It's a renewable and sustainable source of energy.
3. Solar panels convert sunlight to electricity using photovoltaic cells.
4. The amount of sunlight varies by location, affecting efficiency.
5. Solar power can be used for heating, cooling, and generating electricity.


## Chain Runnables (Chain Multiple Runnables)
- We can even **combine this chain** with more runnables to create another chain.
- Let's see how easy our generated output is?

In [52]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='Tell me about {topics} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')
| StrOutputParser()

### Option 1: Using Existing Response

In [53]:
analysis_prompt = ChatPromptTemplate.from_template(
    """
    Analyze the following text: {text}
    
    You need tell me that how difficult it is to understand.
    Answer in one sentence only.
    """
)

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({
    "text": response
})
print(output)

The text is written in simple language, with concise sentences and basic vocabulary, making it easy to comprehend for readers with a basic understanding of science and technology.


### Option 2: Using Composed Chain

In [54]:
composed_chain = {"text": chain} | analysis_prompt | llm | StrOutputParser()

composed_output = composed_chain.invoke({
    "school": "phd",
    "topics": "solar",
    "points": 5,
})

print(composed_output)

The text is written in simple and concise language, with short paragraphs and straightforward bullet points, making it easy to comprehend.


## Parallel Chain
- Parallel chains are used to **run multiple runnables (assuming all runnables are independent) in parallel**.
- The final return value is **a dictionary** with the result of each value under its appropriate key.

In [55]:
system_template = SystemMessagePromptTemplate.from_template("You are {school} teacher. You answer in short sentences")
question_template = HumanMessagePromptTemplate.from_template("Tell me about {topics} in {points} points")

messages = [system_template, question_template]
template = ChatPromptTemplate(messages=messages)

fact_chain = template | llm | StrOutputParser()
output_fact_chain = fact_chain.invoke({
    "school": "senior high school",
    "topics": "solar system",
    "points": 3
})
print(output_fact_chain)

Here are three key points about the solar system:

1. Our solar system consists of eight planets, including Earth.
2. The sun is the center of our solar system and provides light and heat to the planets.
3. Planets orbit around the sun due to its gravitational pull, which keeps them in their stable positions.


In [56]:
question_template = HumanMessagePromptTemplate.from_template("Write a poem on {topics} in {sentences} sentences.")

messages = [system_template, question_template]
template = ChatPromptTemplate(messages=messages)

poem_chain = template | llm | StrOutputParser()
output_poem_chain = poem_chain.invoke({
    "school": "senior high school",
    "topics": "solar system",
    "sentences": 3
})
print(output_poem_chain)

Eight planets dance around the sun,
Mercury to Neptune, their journey's just begun.
A celestial wonder, our universe has won.


In [57]:
from langchain_core.runnables import RunnableParallel

In [58]:
parallel_chain = RunnableParallel(
    fact=fact_chain, 
    poem=poem_chain
)
parallel_chain

{
  fact: ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='Tell me about {topics} in {points} points'), additional_kwargs={})])
        | ChatOllama(model='llama3.2:latest', base_url='http://localhost:11434')
        | StrOutputParser(),
  poem: ChatPromptTemplate(input_variables=['school', 'sentences', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences'), additional_kwargs={}), HumanMessagePromptT

In [59]:
parallel_chain_output = parallel_chain.invoke({
    "school": "senior high school",
    "topics": "solar system",
    "points": 3,
    "sentences": 3
})

print(parallel_chain_output["fact"])
print("\n\n")
print(parallel_chain_output["poem"])

Here are three key points about the solar system:

1. The solar system consists of eight planets, including Earth.
2. The sun is at the center and holds most of the mass in our solar system.
3. Pluto was reclassified as a dwarf planet in 2006 by the International Astronomical Union (IAU).



The sun shines bright at the center space,
Eight planets orbit around with a steady pace,
From Mercury to Neptune, each one has its own place.


## Router Chain
- The router chain is used to **route the output of a previous runnable** to the next runnable based on the output of the previous runnable.

### General Chain to classify either being about positive or negative review

In [60]:
prompt = """Given the user review below, classify it as either being about 'Positive' or 'Negative'.
            Do not respond with more than one word.
            
            Review: {review}
            Classification:"""

general_template = ChatPromptTemplate.from_template(prompt)
general_chain = general_template | llm | StrOutputParser()

review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
general_chain.invoke({
    "review": review
})

'Positive'

### Positive Chain to handle positive review

In [61]:
positive_prompt = """You are expert in writing reply for positive review.
                     You need to encourage the user to share their experience on social media.
                     
                     Review: {review}
                     Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

### Negative Chain to handle negative review

In [62]:
negative_prompt = """You are expert in writing reply for negative review.
                     You need first to apologize for the inconvenience caused to the user.
                     You need to encourage the user to share their concern on following Email: 'cs@company.com'
                     
                     Review: {review}
                     Answer:"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

### Route Function

In [63]:
def route(info):
    if "positive" in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

### Full Chain

In [65]:
from langchain_core.runnables import RunnableLambda

review = "Thank you so much for providing such a great platform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

# RunnableLambda make our method as Langchain Runnable
full_chain = { "sentiment": general_chain, 'review': lambda x: x['review'] } | RunnableLambda(route)
full_chain_output = full_chain.invoke({
    "review": review
})

print(full_chain_output)

Here's a potential reply:

"Dear [Reviewer],

I want to start by sincerely apologizing for the inconvenience you've experienced with our service. We're truly sorry that we haven't met your expectations, and for that, we're deeply regretful.

At [Company Name], we take all feedback seriously and would like to understand more about your concerns. Could you please share some details about what led to your dissatisfaction? Your input will help us identify areas for improvement and work towards providing better services in the future.

If you'd be willing, could you kindly reach out to us directly at cs@company.com so we can discuss this further? We value your feedback and would like to make things right.

Thank you for taking the time to share your experience with us. We look forward to hearing from you soon.

Best regards,
[Your Name]
Customer Support Team"


## Custom Chain with RunnablePassThrough and RunnableLambda
- This is useful for formatting or when you **need functionality not provided** by other Langchain components. and custom function used as Runnables are called RunnableLambdas. 

In [66]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [70]:
def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explain these inputs in 5 sentences: {input1} and {input2}")

In [71]:
custom_chain = prompt | llm | StrOutputParser()
custom_output = custom_chain.invoke({
    "input1": "Earth is planet",
    "input2": "Sun is star",
})

print(custom_output)

The input states that the Earth is a planet, which means it is one of the celestial bodies that orbits around a larger body. The Sun, on the other hand, is identified as a star, indicating that it is a massive ball of hot, glowing gas that provides light and heat to our solar system. In this context, the distinction between a planet and a star highlights their unique characteristics. Planets are typically smaller and less luminous than stars, whereas planets like Earth are mostly composed of rock and metal, while stars like the Sun are primarily made up of hydrogen and helium. This input sets the stage for further discussion about the properties and behaviors of celestial bodies in our solar system.


In [72]:
custom_chain = prompt | llm | StrOutputParser() | {
    'char_counts': RunnableLambda(char_counts),
    'word_counts': RunnableLambda(word_counts),
}

custom_output = custom_chain.invoke({
    "input1": "Earth is planet",
    "input2": "Sun is star",
})

print(custom_output)

{'char_counts': 749, 'word_counts': 129}


In [73]:
custom_chain = prompt | llm | StrOutputParser() | {
    'char_counts': RunnableLambda(char_counts),
    'word_counts': RunnableLambda(word_counts),
    'original_output': RunnablePassthrough() # Using this Runnable causes the original output to appear at the end of chain result
}

custom_output = custom_chain.invoke({
    "input1": "Earth is planet",
    "input2": "Sun is star",
})

print(custom_output)

{'char_counts': 1433, 'word_counts': 232, 'original_output': "Here are the explanations for each input in 5 sentences:\n\n**Earth:** The Earth is a celestial body that orbits around the Sun, making it one of the eight planets in our solar system. It's a rocky, terrestrial planet with a diverse range of landscapes and ecosystems, supporting a vast array of life forms. The Earth's atmosphere is composed of various gases, including oxygen, nitrogen, and carbon dioxide, which play crucial roles in regulating temperature and weather patterns. Our home planet has a solid surface, with mountains, oceans, continents, and forests, making it an incredible world to explore and study. From the majestic Himalayas to the vast Pacific Ocean, Earth is a remarkable and unique planet.\n\n**Sun:** The Sun is the star at the center of our solar system, a massive ball of hot, glowing gas that provides light, heat, and energy to our planet. Composed mostly of hydrogen and helium, the Sun's core is incredibl

## Custom Chain using `@chain` Decorator

In [74]:
from langchain_core.runnables import chain

In [76]:
# The chain decorator will convert the method into a chain
@chain
def custom_chain(params):
    # Parallel Runnable
    return {
        "fact": fact_chain.invoke(params),
        "poem": poem_chain.invoke(params)
    }

params = {
    "school": "senior high school",
    "topics": "big bang",
    "points": 2,
    "sentences": 2
}

output = custom_chain.invoke(params)
print(output['fact'])
print("\n\n")
print(output['poem'])

Here's the Big Bang in 2 points:

1. The universe began with an extremely hot and dense point, approximately 13.8 billion years ago.
2. This singularity rapidly expanded into space, cooling down and forming the stars, galaxies, and planets we see today.



In the beginning, a single point exploded wide,
Creating space and time, the universe did reside.
